# ETF 종목 분석 및 예측을 통한 펀드 개발

## Introduction

Data : 전자공시시스템에서 제공하는 분기별 재무제표 계정과목  
Date : 18년도 4분기(18Y 4Q) ~ 22년도 1분기(22Y 1Q)  
<br>반도체 섹터에 해당하는 134개의 종목 중 46종목 선정

### 패키지 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/krx 공모전/data'

/content/drive/MyDrive/krx 공모전/data


In [ ]:
!ls

catboost_info		 data_0305_20220720.csv  for_test.xlsx	spag2.xlsx
data_0016_20220722.xlsx  data.xlsx		 spag1.xlsx


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from glob import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
#한글 깨짐 처리
plt.rc('font', family='NanumBarunGothic') 

### 데이터 로드

In [ ]:
filst = sorted(glob('*.xlsx'))
filst

['data.xlsx',
 'data_0016_20220722.xlsx',
 'for_test.xlsx',
 'spag1.xlsx',
 'spag2.xlsx']

In [ ]:
data = pd.read_excel(filst[0])
data.head(5)

,분기,종목,자산총계,자본총계,부채총계,매출액,영업이익,당기순이익
0,18Y4Q,유진테크,2.915668e+11,2.634519e+11,2.811489e+10,3.509194e+10,1.068125e+10,2.821543e+08
1,19Y1Q,유진테크,3.069190e+11,2.688406e+11,3.807837e+10,4.282116e+10,1.381908e+10,1.124246e+10
2,19Y2Q,유진테크,3.165687e+11,2.894235e+11,2.714527e+10,6.550883e+10,2.563682e+10,2.077325e+10
3,19Y3Q,유진테크,3.263077e+11,2.984114e+11,2.789634e+10,3.962203e+10,8.644441e+09,9.101949e+09
4,19Y4Q,유진테크,3.185496e+11,2.912752e+11,2.727444e+10,2.623418e+10,-2.851466e+09,-8.300271e+09


## Data processing

### 기업분석 지표<br>
기업분석에 활용되는 지표를 특성별로 2가지씩 사용<br>
- 안정성 : 부채비율 , 자기자본비율  
- 수익성 : 매출액순이익률, 매출액영업이익률  
- 활동성 : 총자산회전율, 자기자본회전율  
- 성장성 : 영업이익증가율, 순이익증가율<br><br>

In [ ]:
#안정성
data['부채비율'] = data['부채총계']/data['자본총계']*100
data['자기자본비율'] = data['자본총계']/data['자산총계']*100
#수익성
data['매출액순이익률'] = data['당기순이익']/data['매출액']*100
data['매출액영업이익률'] = data['영업이익']/ data['매출액'] *100
#활동성
data['총자산회전율'] = data['매출액']/data['자산총계']
data['자기자본회전율']= data['매출액']/ data['자본총계']
#성장성
data['영업이익증가율']=np.NaN
data['순이익증가율'] =np.NaN
for i in range(len(data)-1):
  data['영업이익증가율'][i+1]=(data['영업이익'][i+1]-data['영업이익'][i])/data['영업이익'][i]*100
  data['순이익증가율'][i+1]= (data['당기순이익'][i+1]-data['당기순이익'][i])/data['당기순이익'][i]*100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


### 데이터 프레임 생성  
 
반도체 기업별 주요계정(Major Accounts of the Semiconductor companies, MAS)
- MAS : 18년도 4분기(18Y 4Q) ~ 22년도 1분기(22Y 1Q) 6가지 주요계정  

기업 분석 지표(Business Analysis Indicator, BAI)  
- BAI : 18년도 4분기(18Y 4Q) ~ 22년도 1분기(22Y 1Q) 8가지 기업분석 지표

In [ ]:
mas = data[['분기','종목','자산총계','자본총계','부채총계','매출액','영업이익','당기순이익']]
bai = data[['분기','종목','부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

### 결측치 제거
성장성의 경우, 전 분기 순이익 데이터를 사용하기 때문에 18년도 4분기의 성장성을 구할 수 없음

In [ ]:
#18년도 4분기 제거
index1=bai[bai['분기']=='18Y4Q'].index
bai.drop(index1,inplace=True)
#인덱싱 재배열
bai.reset_index(inplace=True)
bai.drop(['index'],axis=1,inplace=True)
bai.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,분기,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율
0,19Y1Q,유진테크,14.163922,87.593347,26.254461,32.271622,0.139519,0.159281,29.377048,3884.509365
1,19Y2Q,유진테크,9.379083,91.425159,31.710609,39.134906,0.206934,0.226343,85.517542,84.774894


### Data Scaling  
RobustScaler  
이상치의 영향을 최소화하기 위해서 중간값(median)과 사분위값(quartile)을 사용하는 **RobustScaler**를 사용했습니다.

In [ ]:
rb_bai = bai.iloc[:,2:]
rb_bai.head(2)
scaler = RobustScaler()
scaler.fit(rb_bai)
colnames = rb_bai.columns
new_colnames = [i for i in colnames]

In [ ]:
bai_RS = pd.concat([bai.iloc[:,:2],pd.DataFrame(scaler.fit_transform(rb_bai[colnames]),columns=new_colnames)],axis=1)
bai_RS.head(2)

,분기,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율
0,19Y1Q,유진테크,-0.424816,0.578992,0.856453,1.197120,-0.341300,-0.487426,0.328018,30.088515
1,19Y2Q,유진테크,-0.501623,0.713758,1.169686,1.595622,0.220635,-0.185864,0.861672,0.745940


In [ ]:
item = data['종목'].unique()

## Modeling

위에서 얻은 당기 6가지 주요계정(MAS)을 독립변수로, 차기 8가지 기업분석 지표(BAI)를 각각의 종속변수로 하는 8개의 회귀 모델  
<br>독립변수(X) : 19년도 1분기(19Y 1Q) ~ 21년도 3분기(21Y 3Q)
<br>종속변수(Y) : 19년도 2분기(19Y 2Q) ~ 21년도 4분기(21Y 4Q)

### Model 구성을 위한 함수 정의

In [ ]:
def bai_(name,i):
  mas_select = mas[mas['종목']==name]
  bai_select = bai[bai['종목']==name]
  bai_select = bai_select[0:12]
  mas_select_X = mas_select[:12]
  mas_select_x= mas_select[12:13]

  X = mas_select_X.iloc[:,2:8]
  Y = bai_select.iloc[:,i]
  x = mas_select_x.iloc[:,2:8]
  return X,Y,x


#### 안정성  
- 부채비율 
- 자기자본비율 

In [ ]:
# 안정성
# 부채비율
def stability_1(name):
  X,Y,x= bai_(name,2)
  return X,Y,x

# 자기자본비율
def stability_2(name):
  X,Y,x= bai_(name,3)
  return X,Y,x

#### 수익성
- 매출액순이익률
- 매출액영업이익률

In [ ]:
# 수익성
# 매출액순이익률
def profitability_1(name):
  X,Y,x= bai_(name,4)
  return X,Y,x

# 매출액영업이익률
def profitability_2(name):
  X,Y,x= bai_(name,5)
  return X,Y,x

#### 활동성
- 총자산회전율
- 자기자본회전율

In [ ]:
# 활동성
# 총자산회전율
def activity_1(name):
  X,Y,x= bai_(name,6)
  return X,Y,x
# 자기자본회전율
def activity_2(name):
  X,Y,x= bai_(name,7)
  return X,Y,x

#### 성장성
- 영업이익증가율
- 순이익증가율

In [ ]:
# 성장성
# 영업이익증가율
def growth_1(name):
  X,Y,x= bai_(name,8)
  return X,Y,x

# 순이익증가율
def growth_2(name):
  X,Y,x= bai_(name,9)
  return X,Y,x

### Regression

In [ ]:
def linear_model(X,Y,x):
  line_fitter = LinearRegression()
  line_fitter.fit(X,Y)
  y_predicted_s = line_fitter.predict(x)
  return y_predicted_s

def XGB_model(X,Y,x):
  XGB_fitter = XGBRegressor()
  XGB_fitter.fit(X,Y)
  y_predicted_s = XGB_fitter.predict(x)
  return y_predicted_s

def CB_model(X,Y,x):
  CB_fitter = CatBoostRegressor()
  CB_fitter.fit(X,Y)
  y_predicted_s = CB_fitter.predict(x)
  return y_predicted_s

def AB_model(X,Y,x):
  AB_fitter = AdaBoostRegressor()
  AB_fitter.fit(X,Y)
  y_predicted_s = AB_fitter.predict(x)
  return y_predicted_s

def LGBM_model(X,Y,x):
  LGBM_fitter = LGBMRegressor()
  LGBM_fitter.fit(X,Y)
  y_predicted_s = LGBM_fitter.predict(x)
  return y_predicted_s

def GP_model(X,Y,x):
  GP_fitter = GaussianProcessRegressor()
  GP_fitter.fit(X,Y)
  y_predicted_s = GP_fitter.predict(x)
  return y_predicted_s

훈련 세트  
- MAS : 19년도 1분기(19Y 1Q) ~ 21년도 3분기(21Y 3Q) 
- BAI : 19년도 2분기(19Y 2Q) ~ 21년도 4분기(21Y 4Q)   

테스트 세트
- MAS : 21년도 4분기(21Y 4Q) 
- BAI : 22년도 1분기(22Y 1Q)

#### LinearRegression 모델  


In [ ]:
def linear_model_s1(name):
  X,Y,x=stability_1(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

def linear_model_s2(name):
  X,Y,x=stability_2(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

def linear_model_p1(name):
  X,Y,x=profitability_1(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

def linear_model_p2(name):
  X,Y,x=profitability_2(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

def linear_model_a1(name):
  X,Y,x=activity_1(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

def linear_model_a2(name):
  X,Y,x=activity_2(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

def linear_model_g1(name):
  X,Y,x=growth_1(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

def linear_model_g2(name):
  X,Y,x=growth_2(name)
  y_predicted_s=linear_model(X,Y,x)
  return y_predicted_s

##### Model 실행

In [ ]:
s1,s2,p1,p2,a1,a2,g1,g2=[],[],[],[],[],[],[],[]
for i in item :
  s1.append(linear_model_s1(i))
  s2.append(linear_model_s2(i))
  p1.append(linear_model_p1(i))
  p2.append(linear_model_p2(i)) 
  a1.append(linear_model_a1(i))
  a2.append(linear_model_a2(i))
  g1.append(linear_model_g1(i)) 
  g2.append(linear_model_g2(i)) 

##### Data merge

In [ ]:
C= pd.DataFrame(item,columns=['종목'])
S1= pd.DataFrame(s1,columns=['부채비율'])
S2= pd.DataFrame(s2,columns=['자기자본비율'])
P1= pd.DataFrame(p1,columns=['매출액순이익률'])
P2= pd.DataFrame(p2,columns=['매출액영업이익률'])
A1= pd.DataFrame(a1,columns=['총자산회전율'])
A2= pd.DataFrame(a2,columns=['자기자본회전율'])
G1= pd.DataFrame(g1,columns=['영업이익증가율'])
G2= pd.DataFrame(g2,columns=['순이익증가율'])

In [ ]:
predict_LIN = pd.concat([C,S1,S2,P1,P2,A1,A2,G1,G2],axis=1)
predict_LIN = predict_LIN[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

actual_LIN =  bai[(bai['분기']=='22Y1Q')|(bai['분기']== '22Y 1Q')]
actual_LIN.reset_index(inplace=True)
actual_LIN = actual_LIN[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

#### XGBoost

In [ ]:
def XGB_model_s1(name):
  X,Y,x=stability_1(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

def XGB_model_s2(name):
  X,Y,x=stability_2(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

def XGB_model_p1(name):
  X,Y,x=profitability_1(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

def XGB_model_p2(name):
  X,Y,x=profitability_2(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

def XGB_model_a1(name):
  X,Y,x=activity_1(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

def XGB_model_a2(name):
  X,Y,x=activity_2(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

def XGB_model_g1(name):
  X,Y,x=growth_1(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

def XGB_model_g2(name):
  X,Y,x=growth_2(name)
  y_predicted_s=XGB_model(X,Y,x)
  return y_predicted_s

##### Model 실행

In [ ]:
s1,s2,p1,p2,a1,a2,g1,g2=[],[],[],[],[],[],[],[]
for i in item :
  s1.append(XGB_model_s1(i))
  s2.append(XGB_model_s2(i))
  p1.append(XGB_model_p1(i))
  p2.append(XGB_model_p2(i)) 
  a1.append(XGB_model_a1(i))
  a2.append(XGB_model_a2(i))
  g1.append(XGB_model_g1(i)) 
  g2.append(XGB_model_g2(i)) 

[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:20:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

##### Data merge

In [ ]:
C= pd.DataFrame(item,columns=['종목'])
S1= pd.DataFrame(s1,columns=['부채비율'])
S2= pd.DataFrame(s2,columns=['자기자본비율'])
P1= pd.DataFrame(p1,columns=['매출액순이익률'])
P2= pd.DataFrame(p2,columns=['매출액영업이익률'])
A1= pd.DataFrame(a1,columns=['총자산회전율'])
A2= pd.DataFrame(a2,columns=['자기자본회전율'])
G1= pd.DataFrame(g1,columns=['영업이익증가율'])
G2= pd.DataFrame(g2,columns=['순이익증가율'])

In [ ]:
predict_XGB = pd.concat([C,S1,S2,P1,P2,A1,A2,G1,G2],axis=1)
predict_XGB = predict_XGB[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

actual_XGB =  bai[(bai['분기']=='22Y1Q')|(bai['분기']== '22Y 1Q')]
actual_XGB.reset_index(inplace=True)
actual_XGB = actual_XGB[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

#### CatBoost

In [ ]:
def CB_model_s1(name):
  X,Y,x=stability_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_s2(name):
  X,Y,x=stability_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_p1(name):
  X,Y,x=profitability_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_p2(name):
  X,Y,x=profitability_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_a1(name):
  X,Y,x=activity_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_a2(name):
  X,Y,x=activity_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_g1(name):
  X,Y,x=growth_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_g2(name):
  X,Y,x=growth_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

##### Model 실행

In [ ]:
s1,s2,p1,p2,a1,a2,g1,g2=[],[],[],[],[],[],[],[]
for i in item :
  s1.append(CB_model_s1(i))
  s2.append(CB_model_s2(i))
  p1.append(CB_model_p1(i))
  p2.append(CB_model_p2(i)) 
  a1.append(CB_model_a1(i))
  a2.append(CB_model_a2(i))
  g1.append(CB_model_g1(i)) 
  g2.append(CB_model_g2(i)) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 6.5290192	total: 1.43ms	remaining: 286ms
5:	learn: 6.4863239	total: 1.7ms	remaining: 281ms
6:	learn: 6.4197149	total: 1.84ms	remaining: 261ms
7:	learn: 6.3769539	total: 2.09ms	remaining: 259ms
8:	learn: 6.3333071	total: 2.35ms	remaining: 259ms
9:	learn: 6.2929965	total: 2.76ms	remaining: 273ms
10:	learn: 6.2469231	total: 2.94ms	remaining: 265ms
11:	learn: 6.2068127	total: 3.21ms	remaining: 264ms
12:	learn: 6.1783858	total: 3.46ms	remaining: 262ms
13:	learn: 6.1298955	total: 3.71ms	remaining: 261ms
14:	learn: 6.0810150	total: 3.89ms	remaining: 255ms
15:	learn: 6.0552730	total: 4.05ms	remaining: 249ms
16:	learn: 6.0011055	total: 4.23ms	remaining: 244ms
17:	learn: 5.9627593	total: 4.39ms	remaining: 239ms
18:	learn: 5.9232837	total: 4.63ms	remaining: 239ms
19:	learn: 5.8638826	total: 5.06ms	remaining: 248ms
20:	learn: 5.8291996	total: 5.32ms	remaining: 248ms
21:	learn: 5.7929162	total: 5.65ms	remaining: 251ms
22:	learn: 5.7561179	total: 5.95ms	

#####Data merge

In [ ]:
C= pd.DataFrame(item,columns=['종목'])
S1= pd.DataFrame(s1,columns=['부채비율'])
S2= pd.DataFrame(s2,columns=['자기자본비율'])
P1= pd.DataFrame(p1,columns=['매출액순이익률'])
P2= pd.DataFrame(p2,columns=['매출액영업이익률'])
A1= pd.DataFrame(a1,columns=['총자산회전율'])
A2= pd.DataFrame(a2,columns=['자기자본회전율'])
G1= pd.DataFrame(g1,columns=['영업이익증가율'])
G2= pd.DataFrame(g2,columns=['순이익증가율'])

In [ ]:
predict_CB = pd.concat([C,S1,S2,P1,P2,A1,A2,G1,G2],axis=1)
predict_CB = predict_CB[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

actual_CB =  bai[(bai['분기']=='22Y1Q')|(bai['분기']== '22Y 1Q')]
actual_CB.reset_index(inplace=True)
actual_CB = actual_CB[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

#### AdaBoost

In [ ]:
def AB_model_s1(name):
  X,Y,x=stability_1(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

def AB_model_s2(name):
  X,Y,x=stability_2(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

def AB_model_p1(name):
  X,Y,x=profitability_1(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

def AB_model_p2(name):
  X,Y,x=profitability_2(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

def AB_model_a1(name):
  X,Y,x=activity_1(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

def AB_model_a2(name):
  X,Y,x=activity_2(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

def AB_model_g1(name):
  X,Y,x=growth_1(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

def AB_model_g2(name):
  X,Y,x=growth_2(name)
  y_predicted_s=AB_model(X,Y,x)
  return y_predicted_s

##### Model 실행

In [ ]:
s1,s2,p1,p2,a1,a2,g1,g2=[],[],[],[],[],[],[],[]
for i in item :
  s1.append(AB_model_s1(i))
  s2.append(AB_model_s2(i))
  p1.append(AB_model_p1(i))
  p2.append(AB_model_p2(i)) 
  a1.append(AB_model_a1(i))
  a2.append(AB_model_a2(i))
  g1.append(AB_model_g1(i)) 
  g2.append(AB_model_g2(i)) 

#####Data merge

In [ ]:
C= pd.DataFrame(item,columns=['종목'])
S1= pd.DataFrame(s1,columns=['부채비율'])
S2= pd.DataFrame(s2,columns=['자기자본비율'])
P1= pd.DataFrame(p1,columns=['매출액순이익률'])
P2= pd.DataFrame(p2,columns=['매출액영업이익률'])
A1= pd.DataFrame(a1,columns=['총자산회전율'])
A2= pd.DataFrame(a2,columns=['자기자본회전율'])
G1= pd.DataFrame(g1,columns=['영업이익증가율'])
G2= pd.DataFrame(g2,columns=['순이익증가율'])

In [ ]:
predict_AB = pd.concat([C,S1,S2,P1,P2,A1,A2,G1,G2],axis=1)
predict_AB = predict_AB[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

actual_AB =  bai[(bai['분기']=='22Y1Q')|(bai['분기']== '22Y 1Q')]
actual_AB.reset_index(inplace=True)
actual_AB = actual_AB[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

#### LGBM

In [ ]:
def LGBM_model_s1(name):
  X,Y,x=stability_1(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

def LGBM_model_s2(name):
  X,Y,x=stability_2(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

def LGBM_model_p1(name):
  X,Y,x=profitability_1(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

def LGBM_model_p2(name):
  X,Y,x=profitability_2(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

def LGBM_model_a1(name):
  X,Y,x=activity_1(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

def LGBM_model_a2(name):
  X,Y,x=activity_2(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

def LGBM_model_g1(name):
  X,Y,x=growth_1(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

def LGBM_model_g2(name):
  X,Y,x=growth_2(name)
  y_predicted_s=LGBM_model(X,Y,x)
  return y_predicted_s

##### Model 실행

In [ ]:
s1,s2,p1,p2,a1,a2,g1,g2=[],[],[],[],[],[],[],[]
for i in item :
  s1.append(LGBM_model_s1(i))
  s2.append(LGBM_model_s2(i))
  p1.append(LGBM_model_p1(i))
  p2.append(LGBM_model_p2(i)) 
  a1.append(LGBM_model_a1(i))
  a2.append(LGBM_model_a2(i))
  g1.append(LGBM_model_g1(i)) 
  g2.append(LGBM_model_g2(i)) 

#####Data merge

In [ ]:
C= pd.DataFrame(item,columns=['종목'])
S1= pd.DataFrame(s1,columns=['부채비율'])
S2= pd.DataFrame(s2,columns=['자기자본비율'])
P1= pd.DataFrame(p1,columns=['매출액순이익률'])
P2= pd.DataFrame(p2,columns=['매출액영업이익률'])
A1= pd.DataFrame(a1,columns=['총자산회전율'])
A2= pd.DataFrame(a2,columns=['자기자본회전율'])
G1= pd.DataFrame(g1,columns=['영업이익증가율'])
G2= pd.DataFrame(g2,columns=['순이익증가율'])

In [ ]:
predict_LGBM = pd.concat([C,S1,S2,P1,P2,A1,A2,G1,G2],axis=1)
predict_LGBM = predict_LGBM[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

actual_LGBM =  bai[(bai['분기']=='22Y1Q')|(bai['분기']== '22Y 1Q')]
actual_LGBM.reset_index(inplace=True)
actual_LGBM = actual_LGBM[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

#### Gausian Process

In [ ]:
def GP_model_s1(name):
  X,Y,x=stability_1(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

def GP_model_s2(name):
  X,Y,x=stability_2(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

def GP_model_p1(name):
  X,Y,x=profitability_1(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

def GP_model_p2(name):
  X,Y,x=profitability_2(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

def GP_model_a1(name):
  X,Y,x=activity_1(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

def GP_model_a2(name):
  X,Y,x=activity_2(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

def GP_model_g1(name):
  X,Y,x=growth_1(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

def GP_model_g2(name):
  X,Y,x=growth_2(name)
  y_predicted_s=GP_model(X,Y,x)
  return y_predicted_s

##### Model 실행

In [ ]:
s1,s2,p1,p2,a1,a2,g1,g2=[],[],[],[],[],[],[],[]
for i in item :
  s1.append(GP_model_s1(i))
  s2.append(GP_model_s2(i))
  p1.append(GP_model_p1(i))
  p2.append(GP_model_p2(i)) 
  a1.append(GP_model_a1(i))
  a2.append(GP_model_a2(i))
  g1.append(GP_model_g1(i)) 
  g2.append(GP_model_g2(i)) 

#####Data merge

In [ ]:
C= pd.DataFrame(item,columns=['종목'])
S1= pd.DataFrame(s1,columns=['부채비율'])
S2= pd.DataFrame(s2,columns=['자기자본비율'])
P1= pd.DataFrame(p1,columns=['매출액순이익률'])
P2= pd.DataFrame(p2,columns=['매출액영업이익률'])
A1= pd.DataFrame(a1,columns=['총자산회전율'])
A2= pd.DataFrame(a2,columns=['자기자본회전율'])
G1= pd.DataFrame(g1,columns=['영업이익증가율'])
G2= pd.DataFrame(g2,columns=['순이익증가율'])

In [ ]:
predict_GP = pd.concat([C,S1,S2,P1,P2,A1,A2,G1,G2],axis=1)
predict_GP = predict_GP[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

actual_GP =  bai[(bai['분기']=='22Y1Q')|(bai['분기']== '22Y 1Q')]
actual_GP.reset_index(inplace=True)
actual_GP = actual_GP[['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율']]

### 평가 지표
모델의 성능 평가 지표로 이상치에 민감하지 않은 RMSE를 사용

#### RMSE

실제 값과 예측 값의 차이를 제곱하여 평균한 값인 MSE에 루트를 씌운 값

In [ ]:
from sklearn.metrics import mean_squared_error
def rmse(actual,predict):
  RMSE=[]
  for i in range(len(actual.columns)):
    RMSE_value = np.sqrt(mean_squared_error(actual.iloc[:,i],predict.iloc[:,i]))
    RMSE.append(RMSE_value)
  return RMSE

### 모델별 성능 비교

In [ ]:
rmse(actual_LIN,predict_LIN)

[62.100455322543596,
 14.613384586783809,
 116.05639101435261,
 29.41781271367664,
 0.07589613247697787,
 0.18998699857495785,
 7313.530327000558,
 7774.065645171713]

In [ ]:
rmse(actual_XGB,predict_XGB)

[23.019845788092617,
 5.786297646661445,
 33.01866739576112,
 11.720274289022207,
 0.05923396194042429,
 0.18506354606391026,
 1291.7466203761153,
 801.9698549670719]

In [ ]:
rmse(actual_CB,predict_CB)

[24.596413501982692,
 5.5756984315122695,
 20.82231907103389,
 7.456266746720677,
 0.05016040038388708,
 0.14234079568033978,
 1412.5021612275589,
 809.91087353786]

In [ ]:
rmse(actual_AB,predict_AB)

[29.496719752552732,
 5.636551097035596,
 23.787716397961876,
 7.821200872123327,
 0.06189296785077685,
 0.18045694687985978,
 1296.149609863388,
 1076.3201317598766]

In [ ]:
rmse(actual_LGBM,predict_LGBM)

[40.84135679821055,
 8.070284134563964,
 24.75492617441947,
 10.147990096607142,
 0.05793649768692242,
 0.17515070569196517,
 2781.312708756227,
 1899.705265229838]

In [ ]:
rmse(actual_GP,predict_GP)

[66.02657024897285,
 70.89902016255765,
 22.275529409098926,
 20.943805075146443,
 0.27863772026548034,
 0.5021693135933427,
 1470.4740855065243,
 491.8093883331775]

## 최종 모델 결정 : CatBoost

RMSE 값의 평균이 제일 낮은 CatBoost 모델로 최종 선정

<br>독립변수(X) : 19년도 1분기(19Y 1Q) ~ 21년도 4분기(21Y 4Q) 주요계정(MAS)
<br>종속변수(Y) : 19년도 2분기(19Y 2Q) ~ 22년도 1분기(22Y 1Q) 기업분석 지표(BAI)

최종 예측 변수 : 22년도 2분기(22Y 2Q) 기업분석 지표(BAI)

### Model 구성을 위한 함수 정의

In [ ]:
def bai_F(name,i):
  mas_select = mas[mas['종목']==name]
  bai_select = bai[bai['종목']==name]
  bai_select = bai_select[0:13]
  mas_select_X = mas_select[:13]
  mas_select_x= mas_select[13:14]

  X = mas_select_X.iloc[:,2:8]
  Y = bai_select.iloc[:,i]
  x = mas_select_x.iloc[:,2:8]
  return X,Y,x

#### 안정성  
- 부채비율 
- 자기자본비율 

In [ ]:
# 안정성
# 부채비율
def stability_1(name):
  X,Y,x= bai_F(name,2)
  return X,Y,x


# 자기자본비율
def stability_2(name):
  X,Y,x= bai_F(name,3)
  return X,Y,x

#### 수익성
- 매출액순이익률
- 매출액영업이익률

In [ ]:
# 수익성
# 매출액순이익률
def profitability_1(name):
  X,Y,x= bai_F(name,4)
  return X,Y,x

# 매출액영업이익률
def profitability_2(name):
  X,Y,x= bai_F(name,5)
  return X,Y,x

#### 활동성
- 총자산회전율
- 자기자본회전율

In [ ]:
# 활동성
# 총자산회전율
def activity_1(name):
  X,Y,x= bai_F(name,6)
  return X,Y,x

# 자기자본회전율
def activity_2(name):
  X,Y,x= bai_F(name,7)
  return X,Y,x

#### 성장성
- 영업이익증가율
- 순이익증가율

In [ ]:
# 성장성
# 영업이익증가율
def growth_1(name):
  X,Y,x= bai_F(name,8)
  return X,Y,x

# 순이익증가율
def growth_2(name):
  X,Y,x= bai_F(name,9)
  return X,Y,x

### Model Training

In [ ]:
def CB_model_s1(name):
  X,Y,x=stability_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_s2(name):
  X,Y,x=stability_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_p1(name):
  X,Y,x=profitability_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_p2(name):
  X,Y,x=profitability_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_a1(name):
  X,Y,x=activity_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_a2(name):
  X,Y,x=activity_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_g1(name):
  X,Y,x=growth_1(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

def CB_model_g2(name):
  X,Y,x=growth_2(name)
  y_predicted_s=CB_model(X,Y,x)
  return y_predicted_s

In [ ]:
s1,s2,p1,p2,a1,a2,g1,g2=[],[],[],[],[],[],[],[]

In [ ]:
for i in item :
  s1.append(CB_model_s1(i))
  s2.append(CB_model_s2(i))
  p1.append(CB_model_p1(i))
  p2.append(CB_model_p2(i)) 
  a1.append(CB_model_a1(i))
  a2.append(CB_model_a2(i))
  g1.append(CB_model_g1(i)) 
  g2.append(CB_model_g2(i)) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 6.7017953	total: 1.4ms	remaining: 278ms
5:	learn: 6.6636678	total: 1.66ms	remaining: 275ms
6:	learn: 6.6203655	total: 1.91ms	remaining: 271ms
7:	learn: 6.5777176	total: 2.2ms	remaining: 272ms
8:	learn: 6.5363754	total: 2.46ms	remaining: 271ms
9:	learn: 6.4995917	total: 2.73ms	remaining: 270ms
10:	learn: 6.4558963	total: 2.96ms	remaining: 266ms
11:	learn: 6.3921492	total: 3.15ms	remaining: 260ms
12:	learn: 6.3473785	total: 3.35ms	remaining: 255ms
13:	learn: 6.2955213	total: 3.6ms	remaining: 254ms
14:	learn: 6.2433511	total: 3.9ms	remaining: 256ms
15:	learn: 6.2025275	total: 4.15ms	remaining: 255ms
16:	learn: 6.1615117	total: 4.47ms	remaining: 259ms
17:	learn: 6.1497575	total: 4.61ms	remaining: 252ms
18:	learn: 6.1096389	total: 4.82ms	remaining: 249ms
19:	learn: 6.0527085	total: 4.97ms	remaining: 244ms
20:	learn: 6.0146419	total: 5.18ms	remaining: 241ms
21:	learn: 5.9761708	total: 5.48ms	remaining: 244ms
22:	learn: 5.9335128	total: 5.76ms	rem

In [ ]:
C= pd.DataFrame(item,columns=['종목'])
S1= pd.DataFrame(s1,columns=['부채비율'])
S2= pd.DataFrame(s2,columns=['자기자본비율'])
P1= pd.DataFrame(p1,columns=['매출액순이익률'])
P2= pd.DataFrame(p2,columns=['매출액영업이익률'])
A1= pd.DataFrame(a1,columns=['총자산회전율'])
A2= pd.DataFrame(a2,columns=['자기자본회전율'])
G1= pd.DataFrame(g1,columns=['영업이익증가율'])
G2= pd.DataFrame(g2,columns=['순이익증가율'])

In [ ]:
final_predict = pd.concat([C,S1,S2,P1,P2,A1,A2,G1,G2],axis=1)
predict = final_predict.copy()
final_predict.insert(loc=0,column='분기',value='22Y2Q')
final_predict.head()

,분기,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율
0,22Y2Q,유진테크,17.035705,85.087288,1.741215,23.562463,0.139913,0.167140,-46.915405,-746.503453
1,22Y2Q,서울반도체,36.516149,73.253821,3.375966,3.847637,0.275764,0.377135,16.388034,307.467420
2,22Y2Q,SFA반도체,72.218908,58.909443,8.910185,10.357662,0.320220,0.546199,11.893303,24.052948
3,22Y2Q,RFHIC,42.979437,70.078084,7.470620,7.550660,0.055649,0.077722,-515.500699,504.940226
4,22Y2Q,덕산네오룩스,15.402580,86.648087,24.449761,27.917325,0.138837,0.160527,-3.880398,-13.390316


##  종합 점수 산출
- final : 예측값(22년도 1분기 BAI) 추가한 데이터프레임


In [ ]:
final = pd.concat([bai,final_predict])
final.reset_index(inplace=True)
final.drop(['index'],axis=1,inplace=True)
final

,분기,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율
0,19Y1Q,유진테크,14.163922,87.593347,26.254461,32.271622,0.139519,0.159281,29.377048,3884.509365
1,19Y2Q,유진테크,9.379083,91.425159,31.710609,39.134906,0.206934,0.226343,85.517542,84.774894
2,19Y3Q,유진테크,9.348283,91.450910,22.971940,21.817258,0.121425,0.132777,-66.281152,-56.184281
3,19Y4Q,유진테크,9.363805,91.437931,-31.639151,-10.869279,0.082355,0.090067,-132.986122,-191.192237
4,20Y1Q,유진테크,11.702848,89.523232,21.802297,14.458665,0.095657,0.106852,-258.714336,-182.217846
...,...,...,...,...,...,...,...,...,...,...
664,22Y2Q,고영,28.075541,78.130141,18.089425,19.837085,0.166281,0.211607,10.823862,-29.060293
665,22Y2Q,하나머티리얼즈,72.745264,57.892487,25.378254,31.793799,0.179203,0.309628,13.309340,16.261408
666,22Y2Q,이녹스첨단소재,41.390842,69.280309,18.159936,24.312998,0.294970,0.421384,22.887198,16.694111
667,22Y2Q,젬백스,13.452617,88.832480,6.049737,3.444529,0.094344,0.107938,41.733906,-456.176051


### 가중 평균(Weighted Average, wa)
- 19년도 1분기(19Y 1Q) ~ 22년도 2분기(22Y 2Q) BAI<br>
- 가중치 : 1.0, 1.25, 1.5, ... , 4.25


In [ ]:
wa=pd.DataFrame(columns=['종목','부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율'])
for i in item:
  df=final[final['종목']==i]
  df.reset_index(inplace=True) 
  df.drop(['종목','분기','index'],axis=1,inplace=True)
  q,w=0,1
  result=0
  for j in range(14):
    q=df.iloc[j]*w
    w+=0.25
    result+=q
  result = result/36.75
  result['종목']= i
  wa=wa.append(result,ignore_index=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
wa.head()

,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율
0,유진테크,14.969826,87.005395,12.012685,25.397111,0.161108,0.186622,31.709732,-322.114546
1,서울반도체,36.590556,73.266570,3.490394,3.646788,0.259249,0.353776,48.324923,150.670830
2,SFA반도체,88.696028,54.182174,5.283235,7.214493,0.284399,0.526288,25.072035,-16.592978
3,RFHIC,37.792085,72.954948,4.808508,0.250939,0.057379,0.078438,-358.214679,162.849925
4,덕산네오룩스,14.687160,87.207030,23.642065,26.685660,0.162613,0.186415,10.678061,12.109090


In [ ]:
predict.head()

,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율
0,유진테크,17.035705,85.087288,1.741215,23.562463,0.139913,0.167140,-46.915405,-746.503453
1,서울반도체,36.516149,73.253821,3.375966,3.847637,0.275764,0.377135,16.388034,307.467420
2,SFA반도체,72.218908,58.909443,8.910185,10.357662,0.320220,0.546199,11.893303,24.052948
3,RFHIC,42.979437,70.078084,7.470620,7.550660,0.055649,0.077722,-515.500699,504.940226
4,덕산네오룩스,15.402580,86.648087,24.449761,27.917325,0.138837,0.160527,-3.880398,-13.390316


### 종합 점수 산출 방식
- 예측값(22Y 2Q BAI)과 가중 평균값(wa) 비교<br>
 - 부채 비율<br>
   - wa의 110%이하 : 1
   - wa의 110%초과 120%이하 : 0
   - wa의 120%초과 : -1
 - 나머지 BAI
   - wa의 90%이상 : 1
   - wa의 80%이상 90%미만 : 0
   - wa의 80%미만 : -1


In [ ]:
score1=pd.DataFrame()

#### 부채비율

In [ ]:
result1 =predict.iloc[:,1]-wa.iloc[:,1]*1.1
result2 =predict.iloc[:,1]-wa.iloc[:,1]*1.2
a1=[]
for i in range(46):
  if result1[i]<=0:
    a1.append(1)
  elif result2[i] > 0:
    a1.append(-1)
  else: a1.append(0)
score1[wa.columns[1]]=a1

#### 나머지 BAI

In [ ]:
for i in range(2,9):
  result1 =predict.iloc[:,i]-wa.iloc[:,i]*0.9
  result2 =predict.iloc[:,i]-wa.iloc[:,i]*0.8
  a2=[]
  for j in range(46):
    if result1[j]>=0:
      a2.append(1)
    elif result2[j] <0 :
      a2.append(-1)
    else: a2.append(0)
  score1[wa.columns[i]]=a2
score = pd.concat([C,score1],axis=1)

### 산출 결과

In [ ]:
score['총점']=score.iloc[:,1]+score.iloc[:,2]+score.iloc[:,3]+score.iloc[:,4]+score.iloc[:,5]+score.iloc[:,6]+score.iloc[:,7]+score.iloc[:,8]
score.head()

,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율,총점
0,유진테크,0,1,-1,1,0,0,-1,-1,-1
1,서울반도체,1,1,1,1,1,1,-1,1,6
2,SFA반도체,1,1,1,1,1,1,-1,1,6
3,RFHIC,0,1,1,1,1,1,-1,1,5
4,덕산네오룩스,1,1,1,1,0,0,-1,-1,2


## 포트폴리오 구성
- 선정된 종목(34종목)의 종합 점수 비율로 구성 비중 결정
- 종목별로 0.2%p 차감 후 소수점 둘째 자리에서 반올림
- 나머지는 위험 대비 현금으로 구성

#### 포트폴리오 구성을 위한 데이터프레임 생성

In [ ]:
pf = score[score['총점']/8>=0.5]
pf.reset_index(inplace=True)
pf.drop(['index'],axis=1,inplace=True)
pf.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율,총점
0,서울반도체,1,1,1,1,1,1,-1,1,6
1,SFA반도체,1,1,1,1,1,1,-1,1,6
2,RFHIC,0,1,1,1,1,1,-1,1,5
3,테스,1,1,1,1,1,1,1,-1,6
4,미코,1,1,1,1,-1,-1,1,1,4


#### 종목별 비율 설정

In [ ]:
ratio = pf['총점']/sum(pf['총점']) *100
ra=[]
for i in range(len(ratio)):
  ra.append(round(ratio[i]-0.2,1))

In [ ]:
pf['구성비율']=ra
pf.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,종목,부채비율,자기자본비율,매출액순이익률,매출액영업이익률,총자산회전율,자기자본회전율,영업이익증가율,순이익증가율,총점,구성비율
0,서울반도체,1,1,1,1,1,1,-1,1,6,2.8
1,SFA반도체,1,1,1,1,1,1,-1,1,6,2.8
2,RFHIC,0,1,1,1,1,1,-1,1,5,2.3
3,테스,1,1,1,1,1,1,1,-1,6,2.8
4,미코,1,1,1,1,-1,-1,1,1,4,1.8


#### 불필요한 정보 삭제

In [ ]:
pf.drop(['부채비율','자기자본비율','매출액순이익률','매출액영업이익률','총자산회전율','자기자본회전율','영업이익증가율','순이익증가율','총점'],axis=1, inplace=True)
pf.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,종목,구성비율
0,서울반도체,2.8
1,SFA반도체,2.8
2,RFHIC,2.3
3,테스,2.8
4,미코,1.8


#### 현금 비율 설정

In [ ]:
cash_rate = 100-sum(pf['구성비율'])
cash_rate
pf.loc[34]=['현금', cash_rate]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


#### 포트폴리오 완성

In [ ]:
pf.sort_values('구성비율',ascending=False, inplace=True)
pf=pf.reset_index(drop=True)
pf

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,종목,구성비율
0,현금,7.3
1,젬백스,3.9
2,하나머티리얼즈,3.9
3,심텍,3.9
4,엘비세미콘,3.9
5,코미코,3.9
6,티에스이,3.9
7,유니퀘스트,3.9
8,주성엔지니어링,3.9
9,에스앤에스텍,3.4
